In [300]:
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import filedialog


# Open a file selection dialog and get the selected file's path
root = tk.Tk()
root.withdraw()
file_path = filedialog.askopenfilename()

# Load the dataframe
main_df = pd.read_excel(file_path, sheet_name="GISLeaseUpdateResults")
main_df.head(5)

,Internal ID,Lease Type,Lease Subtype,Transaction Number,Lessee(s),Legacy Lease Number,Start Date (Letter Merge),End Date (Letter Merge),Related Asset,Name,Acreage,Legal Description,Lease Terms (Years),Administrator,District,Meridian,Township,Range,Section#,Lease Status
0,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002093,6-5S-59W-10-Surface-Arapahoe-School,70.00,E2,15.0,Rachel Turner,North Central,6,5S,59W,10,Active
1,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002094,6-5S-59W-36-Surface-Arapahoe-School,640.00,ALL,15.0,Rachel Turner,North Central,6,5S,59W,36,Active
2,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002093,6-5S-59W-10-Surface-Arapahoe-School,250.00,E2,15.0,Rachel Turner,North Central,6,5S,59W,10,Active
3,8704,Agriculture,Grazing,AG-100014,"NATURE CONSERVANCY, DICK TANNER, SANDRA J TANN...",AG 42264,2006-11-27,2023-02-27,FAM004437,6-15S-62W-1-Surface-El Paso-School,637.12,"LTS 1-2, S2NE, SE, W2",25.0,Justin Osborne,South Central,6,15S,62W,1,Active
4,8704,Agriculture,Grazing,AG-100014,"NATURE CONSERVANCY, DICK TANNER, SANDRA J TANN...",AG 42264,2006-11-27,2023-02-27,FAM004635,6-16S-62W-1-Surface-El Paso-School,609.18,"LTS 1-4, S2N2, S2",25.0,Justin Osborne,South Central,6,16S,62W,1,Active


In [301]:
# Change Column Names
column_name_changes = {'Internal ID': 'Netsuite Lease ID',
                      'Start Date (Letter Merge)': 'Start Date',
                      'End Date (Letter Merge)': 'End Date',
                      'Related Asset': 'Related FAM(s)',
                      'Name': 'Related FAM Name',
                      'Legal Description': 'Lease Legal Description',
                      'Section#': 'Section'}

# Use the rename method to change the column names
main_df = main_df.rename(columns=column_name_changes)

main_df.head(5)

,Netsuite Lease ID,Lease Type,Lease Subtype,Transaction Number,Lessee(s),Legacy Lease Number,Start Date,End Date,Related FAM(s),Related FAM Name,Acreage,Lease Legal Description,Lease Terms (Years),Administrator,District,Meridian,Township,Range,Section,Lease Status
0,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002093,6-5S-59W-10-Surface-Arapahoe-School,70.00,E2,15.0,Rachel Turner,North Central,6,5S,59W,10,Active
1,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002094,6-5S-59W-36-Surface-Arapahoe-School,640.00,ALL,15.0,Rachel Turner,North Central,6,5S,59W,36,Active
2,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002093,6-5S-59W-10-Surface-Arapahoe-School,250.00,E2,15.0,Rachel Turner,North Central,6,5S,59W,10,Active
3,8704,Agriculture,Grazing,AG-100014,"NATURE CONSERVANCY, DICK TANNER, SANDRA J TANN...",AG 42264,2006-11-27,2023-02-27,FAM004437,6-15S-62W-1-Surface-El Paso-School,637.12,"LTS 1-2, S2NE, SE, W2",25.0,Justin Osborne,South Central,6,15S,62W,1,Active
4,8704,Agriculture,Grazing,AG-100014,"NATURE CONSERVANCY, DICK TANNER, SANDRA J TANN...",AG 42264,2006-11-27,2023-02-27,FAM004635,6-16S-62W-1-Surface-El Paso-School,609.18,"LTS 1-4, S2N2, S2",25.0,Justin Osborne,South Central,6,16S,62W,1,Active


In [302]:
# Group the dataframe by the Lease Number column and sum the values in the Acreage column
acreage_df = main_df.groupby('Transaction Number')['Acreage'].sum().reset_index()

# Rename the Acreage column to Total Lease Acreage
acreage_df = acreage_df.rename(columns={'Acreage': 'Total Lease Acreage'})

# The summary dataframe now contains the total acreage for each Lease Number
acreage_df

,Transaction Number,Total Lease Acreage
0,AG-100001,960.000
1,AG-100014,47859.790
2,AG-100112,2660.550
3,AG-100145,1280.000
4,AG-100148,5175.020
...,...,...
4176,TOW-112838,40.000
4177,TOW-113440,0.028
4178,TOW-113960,160.000
4179,TOW-114153,0.370


In [303]:
main_df = pd.merge(main_df, acreage_df, on='Transaction Number')

main_df.head(5)

,Netsuite Lease ID,Lease Type,Lease Subtype,Transaction Number,Lessee(s),Legacy Lease Number,Start Date,End Date,Related FAM(s),Related FAM Name,...,Lease Legal Description,Lease Terms (Years),Administrator,District,Meridian,Township,Range,Section,Lease Status,Total Lease Acreage
0,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002093,6-5S-59W-10-Surface-Arapahoe-School,...,E2,15.0,Rachel Turner,North Central,6,5S,59W,10,Active,960.00
1,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002094,6-5S-59W-36-Surface-Arapahoe-School,...,ALL,15.0,Rachel Turner,North Central,6,5S,59W,36,Active,960.00
2,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002093,6-5S-59W-10-Surface-Arapahoe-School,...,E2,15.0,Rachel Turner,North Central,6,5S,59W,10,Active,960.00
3,8704,Agriculture,Grazing,AG-100014,"NATURE CONSERVANCY, DICK TANNER, SANDRA J TANN...",AG 42264,2006-11-27,2023-02-27,FAM004437,6-15S-62W-1-Surface-El Paso-School,...,"LTS 1-2, S2NE, SE, W2",25.0,Justin Osborne,South Central,6,15S,62W,1,Active,47859.79
4,8704,Agriculture,Grazing,AG-100014,"NATURE CONSERVANCY, DICK TANNER, SANDRA J TANN...",AG 42264,2006-11-27,2023-02-27,FAM004635,6-16S-62W-1-Surface-El Paso-School,...,"LTS 1-4, S2N2, S2",25.0,Justin Osborne,South Central,6,16S,62W,1,Active,47859.79


In [304]:
# Groupby Lease Legal Descriptions by Transaction number. 

df_legals = main_df.groupby('Transaction Number')['Lease Legal Description'].apply(lambda x: ', '.join([str(i) for i in x])).reset_index()
df_legals.columns = ['Transaction Number', 'Lease Legal Description']

# Nan values need to be dropped
df_legals['Lease Legal Description'] = df_legals['Lease Legal Description'].str.replace('nan, ', '')
df_legals['Lease Legal Description'] = df_legals['Lease Legal Description'].str.replace('nan', '')
df_legals['Lease Legal Description'] = df_legals['Lease Legal Description'].str.replace(', nan', '')

# Replace values with "See Lease Documentation" if greater than 50
mask = df_legals['Lease Legal Description'].str.len() > 50
df_legals['Lease Legal Description'] = df_legals['Lease Legal Description'].where(~mask, 'See Lease Documentation')
df_legals = df_legals.rename(columns={'Lease Legal Description': 'Lease Legal Description MERGED'})

df_legals.sample(25)

,Transaction Number,Lease Legal Description MERGED
3758,SM-102433,"SW/4SE/4, SW/4SE/4, SW/4SE/4, SW/4SE/4"
222,AG-101888,ALL
731,AG-109419,"E2W2, LOTS 1,2"
3920,SM-102734,S2/NW4; N/2SE/4
742,AG-109476,See Lease Documentation
3215,RAP-105686,See Lease Documentation
2637,OG-108487,"ALL, ALL, ALL, ALL, ALL, ALL, ALL,"
1086,AG-110367,"N2S2,"
1645,AG-112928,"ALL, ALL,"
1116,AG-110422,"SW, W2SE, W2NE, W2, W2SE, W2NE, N2NW, SENW, ALL,"


In [305]:
# Merge main_df and df_legals into merged_df
main_df = pd.merge(main_df, df_legals, on='Transaction Number')
if 'Lease Legal Description MERGED' in main_df.columns:
    main_df.rename(columns={'Lease Legal Description MERGED': 'Lease Legal Description'}, inplace=True)


main_df.head(5)

,Netsuite Lease ID,Lease Type,Lease Subtype,Transaction Number,Lessee(s),Legacy Lease Number,Start Date,End Date,Related FAM(s),Related FAM Name,...,Lease Terms (Years),Administrator,District,Meridian,Township,Range,Section,Lease Status,Total Lease Acreage,Lease Legal Description
0,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002093,6-5S-59W-10-Surface-Arapahoe-School,...,15.0,Rachel Turner,North Central,6,5S,59W,10,Active,960.00,"E2, ALL, E2"
1,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002094,6-5S-59W-36-Surface-Arapahoe-School,...,15.0,Rachel Turner,North Central,6,5S,59W,36,Active,960.00,"E2, ALL, E2"
2,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002093,6-5S-59W-10-Surface-Arapahoe-School,...,15.0,Rachel Turner,North Central,6,5S,59W,10,Active,960.00,"E2, ALL, E2"
3,8704,Agriculture,Grazing,AG-100014,"NATURE CONSERVANCY, DICK TANNER, SANDRA J TANN...",AG 42264,2006-11-27,2023-02-27,FAM004437,6-15S-62W-1-Surface-El Paso-School,...,25.0,Justin Osborne,South Central,6,15S,62W,1,Active,47859.79,See Lease Documentation
4,8704,Agriculture,Grazing,AG-100014,"NATURE CONSERVANCY, DICK TANNER, SANDRA J TANN...",AG 42264,2006-11-27,2023-02-27,FAM004635,6-16S-62W-1-Surface-El Paso-School,...,25.0,Justin Osborne,South Central,6,16S,62W,1,Active,47859.79,See Lease Documentation


In [306]:
main_df.fillna("",inplace=True)
section_df = main_df.groupby('Transaction Number')['Section'].apply(lambda x: ', '.join(filter(None, [val.strip() for val in x])) if x.nunique()>1 else x.iloc[0]).reset_index()

# Replace values with "See Lease Documentation" if greater than 20
mask = section_df['Section'].str.len() > 20
section_df['Section'] = section_df['Section'].where(~mask, 'See Lease Documentation')

section_df.sample(25)

,Transaction Number,Section
811,AG-109684,"17, 36, 36, 17"
2626,OG-108476,See Lease Documentation
106,AG-101755,See Lease Documentation
3021,OG-113586,10
3418,REC-112237,16
362,AG-102045,See Lease Documentation
4055,TAP-110359,14
3828,SM-102598,"11, 14, 15"
1268,AG-111020,6
3505,REC-113096,"1, 2, 11, 12, 36"


In [307]:
# Rename Section to Section(s) in main_df
section_df.rename(columns={'Section': 'Section(s)'}, inplace=True)
main_df = pd.merge(main_df, section_df[['Transaction Number', 'Section(s)']], on='Transaction Number', how='left')

# Check if Section is dropped
if 'Section' in main_df.columns:
    main_df.drop(columns=['Section'], inplace=True)


main_df.head(5)


,Netsuite Lease ID,Lease Type,Lease Subtype,Transaction Number,Lessee(s),Legacy Lease Number,Start Date,End Date,Related FAM(s),Related FAM Name,...,Lease Terms (Years),Administrator,District,Meridian,Township,Range,Lease Status,Total Lease Acreage,Lease Legal Description,Section(s)
0,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002093,6-5S-59W-10-Surface-Arapahoe-School,...,15.0,Rachel Turner,North Central,6,5S,59W,Active,960.00,"E2, ALL, E2","10, 36, 10"
1,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002094,6-5S-59W-36-Surface-Arapahoe-School,...,15.0,Rachel Turner,North Central,6,5S,59W,Active,960.00,"E2, ALL, E2","10, 36, 10"
2,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002093,6-5S-59W-10-Surface-Arapahoe-School,...,15.0,Rachel Turner,North Central,6,5S,59W,Active,960.00,"E2, ALL, E2","10, 36, 10"
3,8704,Agriculture,Grazing,AG-100014,"NATURE CONSERVANCY, DICK TANNER, SANDRA J TANN...",AG 42264,2006-11-27,2023-02-27,FAM004437,6-15S-62W-1-Surface-El Paso-School,...,25.0,Justin Osborne,South Central,6,15S,62W,Active,47859.79,See Lease Documentation,See Lease Documentation
4,8704,Agriculture,Grazing,AG-100014,"NATURE CONSERVANCY, DICK TANNER, SANDRA J TANN...",AG 42264,2006-11-27,2023-02-27,FAM004635,6-16S-62W-1-Surface-El Paso-School,...,25.0,Justin Osborne,South Central,6,16S,62W,Active,47859.79,See Lease Documentation,See Lease Documentation


In [308]:
# Check if Section is dropped
if 'Section' in main_df.columns:
    main_df.drop(columns=['Section'], inplace=True)


main_df.head(5)


,Netsuite Lease ID,Lease Type,Lease Subtype,Transaction Number,Lessee(s),Legacy Lease Number,Start Date,End Date,Related FAM(s),Related FAM Name,...,Lease Terms (Years),Administrator,District,Meridian,Township,Range,Lease Status,Total Lease Acreage,Lease Legal Description,Section(s)
0,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002093,6-5S-59W-10-Surface-Arapahoe-School,...,15.0,Rachel Turner,North Central,6,5S,59W,Active,960.00,"E2, ALL, E2","10, 36, 10"
1,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002094,6-5S-59W-36-Surface-Arapahoe-School,...,15.0,Rachel Turner,North Central,6,5S,59W,Active,960.00,"E2, ALL, E2","10, 36, 10"
2,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002093,6-5S-59W-10-Surface-Arapahoe-School,...,15.0,Rachel Turner,North Central,6,5S,59W,Active,960.00,"E2, ALL, E2","10, 36, 10"
3,8704,Agriculture,Grazing,AG-100014,"NATURE CONSERVANCY, DICK TANNER, SANDRA J TANN...",AG 42264,2006-11-27,2023-02-27,FAM004437,6-15S-62W-1-Surface-El Paso-School,...,25.0,Justin Osborne,South Central,6,15S,62W,Active,47859.79,See Lease Documentation,See Lease Documentation
4,8704,Agriculture,Grazing,AG-100014,"NATURE CONSERVANCY, DICK TANNER, SANDRA J TANN...",AG 42264,2006-11-27,2023-02-27,FAM004635,6-16S-62W-1-Surface-El Paso-School,...,25.0,Justin Osborne,South Central,6,16S,62W,Active,47859.79,See Lease Documentation,See Lease Documentation
